<a href="https://colab.research.google.com/github/marshallarnold/Fall-Detection-Image-Classifier/blob/main/falldetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/valid'

train_datagen = ImageDataGenerator(rescale=1/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255)

valid_datagen = ImageDataGenerator(rescale=1/255)

batch_size = 32
class_mode = 'binary'

train_data = train_datagen.flow_from_directory(train_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode)
test_data = test_datagen.flow_from_directory(test_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode, shuffle = False)
valid_data = valid_datagen.flow_from_directory(valid_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode)

Found 909 images belonging to 2 classes.
Found 51 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


In [5]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape = (224,224,3)),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

In [9]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_data, epochs=10, validation_data=valid_data, callbacks=[early_stop])

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.9058 - loss: 0.1959 - val_accuracy: 1.0000 - val_loss: 0.0844
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9935 - loss: 0.0127 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.9961 - loss: 0.0132 - val_accuracy: 1.0000 - val_loss: 1.5374e-04
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.9969 - loss: 0.0060 - val_accuracy: 1.0000 - val_loss: 8.8350e-06
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.9948 - loss: 0.0083 - val_accuracy: 1.0000 - val_loss: 4.4910e-07
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.9988 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 1.6529e-07
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 1.0000 - loss: 7.5142e-04 - val_accuracy: 1.0000 - val_loss: 5.9126e-08
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 1.0000 - loss: 6.4026e-04 - val_